Author: _Rafieh Mosaheb_

Date: 24 Sep 2020
***
This .ipynb code solves task 1 with computing the gradient of objective function 

\begin{equation}
    f(\theta):=||\psi(\theta)-\phi||=<\psi(\theta)-\phi|\psi(\theta)-\phi>=2-<\psi(\theta)|\phi>-<\phi|\psi(\theta)>
\end{equation}

and using the common gradient discent formula: <font color=blue>$\theta_{i+1}=\theta_i - \eta \nabla f(\theta_i)$</font> where $\theta_i$ is the vector of $8\times L$ angles in the interval $(0,4\pi)$, where $L$ is the number of layers in the circuit.

In [92]:
import matplotlib.pyplot as plt
from array import *
from numpy import linalg as LA
import numpy as np
import sympy
from sympy.physics.quantum.dagger import Dagger
import random
%matplotlib inline

For the sake of easiness, I define my own Kronecker product of 4 matrices:

In [93]:
#Kronecker product on 4 matrices
def rafiehKron(M1, M2, M3, M4):
    rK = np.kron(M1,M2)
    rK = np.kron(rK,M3)
    rK = np.kron(rK,M4)
    return rK

Then, I define all the necessary scalar, unitaries, and vectors that I need to work with. According to the following equalities, we have:
$$\text{ket04}=|0>^{\otimes 4},\quad \text{ketbra0}=|0><0|,\quad \text{ketbra1}=|1><1|$$

In [94]:
pi = np.pi
im = sympy.I
e = np.e
I = np.diag([1, 1])
Z = np.diag([1, -1])
X = np.matrix([[0,1],[1,0]])
ZZ = np.kron(np.diag([1, -1]), np.diag([1, -1]))
XX = np.kron([[0, 1],[1, 0]], [[0, 1],[1, 0]])
ket04 = rafiehKron(np.matrix([[1],[0]]),np.matrix([[1],[0]]),np.matrix([[1],[0]]),np.matrix([[1],[0]]))
ketbra0 = np.matrix([[1,0],[0,0]])
ketbra1 = np.matrix([[0,0],[0,1]])

In the next cell, $gate$ is the Pauli matrix $Z$ or $X$, as we see in the circuit, and $angle$ is the rotation angle about the Pauli axis in the Bloch sphere. Hence, 
\begin{equation}
    \text{R(gate, angle)}:= e^{-i\ angle/2\ gate}=\cos(angle/2)\ \text{I}-i\sin(angle/2)\ gate
\end{equation}

In [95]:
def R(gate, angle):
    R = np.cos(angle/2) * I - im * np.sin(angle/2) * gate            
    return R

Now we want to compute the Unitary corresponding to the unparametrized gates of the circuit, controlled-Z's, which are included in the even blocks ($U_{\text{even}}$). To this end, we need to discover the $control$ and $target$ qubits in each of the 6 c-Z's and then use the following formulla:
$$c-Z(control,target) = |0><0|\otimes I + |1><1|\otimes Z,$$
where the first matrix is applied on the $control$ qubit and the second one is applied on the $target$ qubit. The dictionary defined in the next cell is meant to keep the track of this two qubit:

In [96]:
M1 = I
M2 = I
M3 = I
M4 = I

Kron_Dic = { '1':M1, '2':M2, '3':M3, '4':M4}
val_list = list(Kron_Dic.values()) 

The point here is that we need to pay attention to the order of c-Z in the circuit. We consider the ordering of applying them from left to write. Hence the matrix multiplication will be from right to left, as follows:

In [97]:
#Function c_Z: the controlled-Z gate on two qubit
#Input: control --> integer 0,1,2
#       target --> integer 1,2,3
def c_Z(control, target):
    val_list[control] = ketbra0
    U_1 = rafiehKron(val_list[0], val_list[1], val_list[2], val_list[3])
    
    val_list[control] = ketbra1
    val_list[target] = Z
    U_2 = rafiehKron(val_list[0], val_list[1], val_list[2], val_list[3])
    
    return U_1+U_2
 
#Function c_Z_Unitary: the corresponding matrix of applying the 6 c-Z gates in U_{even}    
def c_Z_Unitary():
    U = np.identity(16)
    for control in range(2,-1,-1):
        for target in range(3,control,-1):
            U = U * c_Z(control,target)
    return U        

In the following cell, we define the odd and even blocks $U_i$.Here, we assume that $\theta$ is a reshaped 2D array angle vector. Then, theta[j][k] is the same as angles shown in the circuit: $j$, $1\leq j\leq 2L$, is the block index in $U_j$ and $0\leq k\leq 3$ is the wire.

In [98]:
def Ui(i, theta):
    Ui_mat = np.identity(16,dtype=np.complex64)
    if i%2 == 1:
        Ui_mat = rafiehKron(R(X,theta[i-1][0]), R(X,theta[i-1][1]), R(X,theta[i-1][2]), R(X,theta[i-1][3]))
    else:
        Ui_mat = c_Z_Unitary()     
        Ui_mat = Ui_mat * rafiehKron(R(Z,theta[i-1][0]), R(Z,theta[i-1][1]), R(Z,theta[i-1][2]), R(Z,theta[i-1][3]))
            
    return Ui_mat

In the following cell, we compute the partial derivative derivU(j, k)$:=\frac{\partial U_j(\theta_j)}{\partial\theta_{j,k}}$:
1. j odd: $\frac{\partial U_j(\theta_j)}{\partial\theta_{j,k}}=\frac{-i}{2}R_x(\theta_{j,1})\otimes\cdots\otimes X\times R_x(\theta_{j,k})\otimes\cdots\otimes R_x(\theta_{j,4})$

2. j even: $\frac{\partial U_j(\theta_j)}{\partial\theta_{j,k}}=\frac{-i}{2}\text{c_Z_Unitary()}R_z(\theta_{j,1})\otimes\cdots\otimes Z\times R_z(\theta_{j,k})\otimes\cdots\otimes R_z(\theta_{j,4})$

In [99]:
#Input: theta --> reshaped 2D array of angles
#       j --> integer 1,2,3,...,2*L which indicates index of Unitaries
#       k --> integer 1,2,3,4 which indicates the wire
def derivU(theta, j, k):
    j -= 1
    k -= 1
    
    gate = I
    mat = rafiehKron(I,I,I,I)
    
    if j%2 == 1:
        gate = X
    else:
        gate = Z
        mat = c_Z_Unitary()
    
    M1 = R(gate,theta[j][0])
    M2 = R(gate,theta[j][1])
    M3 = R(gate,theta[j][2])
    M4 = R(gate,theta[j][3])    
    dUj_Dic = { '1':M1, '2':M2, '3':M3, '4':M4}
    val_list = list(dUj_Dic.values()) 
    val_list[k] = gate * val_list[k]
    dUjk = (-im/2) * mat * rafiehKron(val_list[0],val_list[1],val_list[2],val_list[3]) 
            
    return dUjk

In the following cell, we generate a random vector $\phi$ on 4 qubits. We devide the vector by it's norm in order to demonstrate a valid quantum state.

In [100]:
phi = np.random.random(16) + np.random.random(16) * 1j
phi = [x / LA.norm(phi) for x in phi] 
phi = np.array([phi]).T
bra_phi = phi.conj().T

In the following cell, we compute $\nabla_{j,k}f(\theta)=\frac{\partial f(\theta)}{\partial\theta_{j,k}}=-\frac{\partial <\psi(\theta)|\phi>}{\partial\theta_{j,k}}-\frac{\partial <\phi|\psi(\theta)>}{\partial\theta_{j,k}}$:

In [101]:
#Input: theta --> reshaped 2D array of angles
#       j --> integer 1,2,3,...,2*L which indicates index of Unitaries
#       k --> integer 1,2,3,4 which indicates the wire
#       L --> number of layers in the circuit
def gradient(theta, j, k, L):
    U = np.identity(16)
    for i in range(2*L,j,-1):
        U = U * Ui(i, theta)
    U = U * derivU(theta,j,k)
    for i in range(j-1,0,-1):
        U = U * Ui(i, theta)    
        
    grad1 = bra_phi * U * ket04
    grad2 = Dagger(U * ket04) * phi
    grad = -(grad1+grad2)
    return grad

In the next cell, we compute the gradient vector $\nabla f(\theta)$, for each $j$ and $k$:

In [133]:
#Input: angles --> row array of angles:
def gradf(angles, L):
    grad = np.zeros((2*L,4))
    theta = angles.reshape([2*L,4])
    for j in range(1,2*L+1,+1):
        for k in range(1,5,+1):
            #grad.append(gradient(theta, j, k, L))
            #print(gradient(theta, j, k, L))
            grad[j-1][k-1] = gradient(theta, j, k, L)
    
    return grad.reshape([1,8*L])    

Now, we can start the gradient discent algorithm, starting with some random $\theta_0$ and the following update rule:
$$\theta_{i+1}=\theta_i - \eta \nabla f(\theta_i)$$

In [129]:
eta = 0.5
n_steps = 6

def grad_des_alg(L):
    current_angles = []
    for i in range(8*L):
        #current_angles[i] = random.uniform(0, 4*pi)
        current_angles.append(random.uniform(0, 4*pi))
    for i in range(n_steps):
        ang = np.array(current_angles)
        Gi = gradf(ang, L)
        #update = np.array([x * eta for x in Gi] )
        new_angles = current_angles - eta * Gi
        current_angles = new_angles
    return new_angles

Now is the time to check the algorithm and see the result for desired $\eta$, $n_{\text{steps}}$, and $L$:

In [ ]:
minTheta = grad_des_alg(3)